### PLEASE COPY NOTEBOOKS TO YOUR FOLDERS TO PREVENT COMMIT CONFLICTS
* If you would like to contribute to this notebook, make changes on it in useful_notebooks folder, run "Restart and Clear Output" before commit.

#### Unexpected Expansion Cases
###### Experiment set fetches unrealated labs and awards
Sometimes a fetch will get some unrelated labs and awards, this is because of the multiple awards a lab can have. This multiple awards are visited, which have users linked to them. This users also have labs, so here you go. Hopefully all are released/current already.

###### Experiment set fetches unrealted biosmaple/experiment/set
This was so far because of the experiment and biosample relation field, or the references field that links to a publication. If you ignore these fields, if should be fine.
`ignore_field = ['experiment_relation', 'biosample_relation', 'references']`

In [ ]:
from dcicutils import ff_utils
from functions.notebook_functions import *
from functions.cleanup import *
import time
time1 = time.time()

my_auth = get_key('andyprod', keyfile='~/saved_keypairs.json')
#my_auth = get_key('koray_data')

# Please Modify the following accordingly 
# do you want to check for duplicate/problematic runs on files?
# it will take some time
check_wfrs = True
delete_problematic = False

# Which status to change
change_status = 'released'

sets_in_scope = [] # ['4DNACCCC', '4DNACCCCC']

search_url  = '/search/?award.project=4DN&experiments_in_set.experiment_type=dilution+Hi-C&experimentset_type=replicate&lab.display_title=Bing+Ren%2C+UCSD&status=pre-release&type=ExperimentSetReplicate'

if sets_in_scope:
    set_to_release = [ff_utils.get_metadata(i, my_auth)['uuid'] for i in sets_in_scope]
elif search_url:
    set_to_release = [i['uuid'] for i in ff_utils.search_metadata(search_url, my_auth)]

store={}
item_uuids=[]
store, uuids = ff_utils.expand_es_metadata(set_to_release, my_auth, store_frame='embedded',add_pc_wfr=True, ignore_field = ['experiment_relation', 'biosample_relation', 'references'])

print(len(store['experiment_set_replicate']), 'exp sets for status change')
print(len(uuids), 'items collected')
time2 = time.time()
print(round((time2-time1), 1), 'sec for collection')

In [ ]:
# TODO
# Check audits

# create stash of wfrs to pass to delete_wfrs
stash = store.get('workflow_run_sbg', []) + store.get('workflow_run_awsem', [])

# to decide which items to ignore as they have a 'higher' status
STATUS_LEVEL = {
    # standard_status
    "released": 10, "current": 10,
    "released to project": 9,
    "pre-release": 8,
    "planned": 6, "submission in progress": 6,
    "in review by lab": 4,
    "revoked": 0, "archived": 0,"deleted": 0, "obsolete": 0, "replaced": 0, "archived to project": 0,
    # additional file statuses
    'to be uploaded by workflow': 4, 'uploading': 4, 'uploaded': 4, 'upload failed': 4, 'draft': 4, 'released to lab': 4}

change_level = STATUS_LEVEL.get(change_status, 1)

# check expsets
print('EXPSET CHECK')
for a_set in store['experiment_set_replicate']:
    if not a_set.get('completed_processes'):
        print(a_set['accession'], 'missing processing tag', a_set['description'][:50])

# check exps 
print('\nEXP CHECK')
# check for experiment numbers
exp_names = [i for i in store if i.startswith('experiment') and not i.startswith('experiment_set')]
all_exps_on_sets = [a for i in store['experiment_set_replicate'] for a in i['experiments_in_set']]
all_exps = [a['uuid'] for i in store.keys() for a in store[i] if i in exp_names]
if len(all_exps_on_sets) != len(all_exps):
    print('Number of experiments is not same as experiments associated with sets')
    print('# of exps: {}. # of exps on sets: {}'.format(len(all_exps), len(all_exps_on_sets)))
        
print('\nFILE FASTQ CHECK')
for a_file in store['file_fastq']:
    if not a_file.get('quality_metric'):
        print(a_file['accession'], 'missing fastqc')
    if not a_file.get('content_md5sum'):
        print(a_file['accession'], 'missing content md5 sum')
    if not a_file.get('md5sum'):
        print(a_file['accession'], 'md5 was not calculated during upload, missing md5sum')
    if check_wfrs:
        dw = delete_wfrs(a_file, my_auth, delete=delete_problematic, stash=stash)

# check processed files
print('\nFILE PROCESSED CHECK')
if store.get('file_processed'):
    for a_file in store['file_processed']:
        if a_file['file_format']['file_format'] == '/file-formats/pairs/':
            if not a_file.get('quality_metric'):
                print(a_file['accession'], 'missing Pairsqc')
        if not a_file.get('source_experiments'):
            print(a_file['accession'], 'user submitted or produced by sbg runs')
        if check_wfrs:
            dw = delete_wfrs(a_file, my_auth, delete=delete_problematic, stash=stash)   

# check wfrs
print('\nWFR CHECK')
# list all wf types found
print('  Following run types are found:')
for wf in set([i['display_title'].split(' run')[0] for i in store.get('workflow_run_awsem')]):
           print('    ' + wf)
if store.get('workflow_run_awsem'):
    for wfr in store['workflow_run_awsem']:
        if wfr['run_status'] != 'complete':
            print('problematic wfr', wfr['uuid'], wfr['run_status'])
        
# check for weird status
print('\nREPORT NUMBERS AND CHECK STATUS')
for i in store:
    print(i, len(store[i]))
    weird = [[i, x['uuid'], x['status']] for x in store[i] if STATUS_LEVEL.get(x['status']) == 0]
    if weird:
        for case in weird:
            print(case)
        print()

In [ ]:
# Check status
print_each = False
counter = 0
for a_type in store:
    total = len(store[a_type])
    change = 0
    matching = 0
    unusual = 0
    skipping = 0
    for raw_data in store[a_type]:
        if STATUS_LEVEL.get(raw_data['status']) > change_level:
            skipping += 1
            msg = ('{} {} ITEM HAS STATUS {} HIGHER THAN {} - SKIPPING'.format(a_type, raw_data['uuid'], raw_data['status'], change_status))
        elif STATUS_LEVEL.get(raw_data['status']) == STATUS_LEVEL.get(change_status):
            matching += 1
            msg = ('MATCHING ACCESS STATUS', a_type, raw_data['uuid'], raw_data['status'])
        elif STATUS_LEVEL.get(raw_data['status']) == 0:
            unusual += 1
            msg = ('SKIP UNUSUAL STATUS  ', a_type, raw_data['uuid'], raw_data['status'])
        else:
            change += 1
            msg = ('       CHANGE        ', a_type, raw_data['uuid'], raw_data['status'])
        if print_each:
            print(msg)
    print('{:<25} Out of {t}, {r} skipped, {m} matching, {u} unusual, and {c} needs change'.format(a_type, t=total, r=skipping, m=matching, u=unusual, c=change))
    
       

In [ ]:
# If you want to patch the status, change action to True
action = False

reviewed = ""
reviewed = input('Did another wrangler review this release? (y/n):')
if reviewed != 'y':
    raise KeyError('A key step is missing!')
    
print_each = False
counter = 0
for a_type in store:
    total = len(store[a_type])
    change = 0
    matching = 0
    unusual = 0
    skipping = 0
    for raw_data in store[a_type]:
        if STATUS_LEVEL.get(raw_data['status']) > change_level:
            skipping += 1
            msg = ('{} {} ITEM HAS STATUS {} HIGHER THAN {} - SKIPPING'.format(a_type, raw_data['uuid'], raw_data['status'], change_status))
        elif STATUS_LEVEL.get(raw_data['status']) == STATUS_LEVEL.get(change_status):
            matching += 1
            msg = ('MATCHING ACCESS STATUS', a_type, raw_data['uuid'], raw_data['status'])
        elif STATUS_LEVEL.get(raw_data['status']) == 0:
            unusual += 1
            msg = ('SKIP UNUSUAL STATUS  ', a_type, raw_data['uuid'], raw_data['status'])
        else:
            change += 1
            msg = ('       CHANGE        ', a_type, raw_data['uuid'], raw_data['status'])
            if action:
                patch_data = {'status': change_status}
                if change_status == 'released' and a_type in ['publication']:
                    patch_data = {'status': 'current'}
                   
                ff_utils.patch_metadata(patch_data, obj_id=raw_data['uuid'] ,key=my_auth) 

        if print_each:
            print(msg)
    print('{:<25} Out of {t}, {r} skipped, {m} matching, {u} unusual, and {c} UPDATED with status'.format(a_type, t=total, r=skipping, m=matching, u=unusual, c=change))